<a href="https://colab.research.google.com/github/JDS289/BaLD4LLM/blob/main/playingAround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using GPU seems to sometimes cause a CUDA error; using TPU seems to sometimes cause "Unknown crash".
CPU takes approximately 10x longer than GPU.


3B has said False to "Paris is the capital of France". Might want to use 8B instead. For now, I'll continue to use 3B for speed/simplicity. This is also justified for "reproducibility" reasons, though if accuracy is negligible then switch to a better model.


In [1]:
from google.colab import userdata
hf_token = userdata.get("huggingface_secret")

from huggingface_hub import login
login(token=hf_token)

In [12]:
DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm

#import warnings
#warnings.filterwarnings('ignore')


# Display first 500 characters of extracted text as preview


device = "cuda" if torch.cuda.is_available() else "cpu"

#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SYS_PROMPT = ""


#Let's load in the model and start processing the text chunks

accelerator = Accelerator()

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id


OSError: meta-llama/Llama-3.2-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [4]:
model.generation_config.output_hidden_states = True
model.generation_config.return_dict_in_generate = True
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "output_hidden_states": true,
  "pad_token_id": 128009,
  "return_dict_in_generate": true,
  "temperature": 0.6,
  "top_p": 0.9
}



In [5]:
def process_chunk(text_chunk):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).strip()
    processed_text = processed_text[processed_text.index("assistant") + 11:]


    """
    for i in range(len(output.hidden_states)):
        total_shape = np.array(output.hidden_states[i][0].shape)
        for j in range(1, len(output.hidden_states[i])):
            total_shape += np.array(output.hidden_states[i][j].shape)
        print(total_shape)"""
    return processed_text, output.hidden_states

In [29]:
import requests

capitals = requests.get("https://raw.githubusercontent.com/icyrockcom/country-capitals/refs/heads/master/data/country-list.csv")
#print(capitals.content.decode("utf-8").split("\n")[1:])
capitals_dict = {}
for string in capitals.content.decode("utf-8").split("\n")[1:]:
  capital, city, _type = string.split(",")
  capitals_dict[capital] = city

print(capitals_dict)


ValueError: too many values to unpack (expected 3)

In [17]:
statement1 = "Paris is the capital of France"
statement2 = "Madrid is the capital of Spain"

response1, hidden_states1 = process_chunk(f'"{statement1}". Is this true or false? (Answer ONLY with "True." or "False.")')
response2, hidden_states2 = process_chunk(f'"{statement2}". Is this true or false? (Answer ONLY with "True." or "False.")')

print(response1)
print(response2)


False.
True.


In [8]:
print(hidden_states)

((tensor([[[-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
          -5.4550e-04,  8.8215e-05],
         [-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
          -5.4550e-04,  8.8215e-05],
         [-1.4160e-02,  1.2390e-02,  8.9264e-04,  ..., -1.6785e-03,
           6.3171e-03, -3.3722e-03],
         ...,
         [-7.9956e-03, -3.9307e-02,  1.2390e-02,  ..., -3.0518e-03,
          -2.0874e-02,  2.9785e-02],
         [ 8.1177e-03, -2.3804e-02, -9.3994e-03,  ..., -4.5776e-03,
           2.2827e-02,  2.4658e-02],
         [-1.5137e-02,  1.0071e-02, -4.9438e-03,  ...,  3.7689e-03,
          -1.1475e-02, -2.3346e-03]]], device='cuda:0', dtype=torch.bfloat16), tensor([[[-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
         [-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
         [-0.0303,  0.0046, -0.0664,  ...,  0.0121, -0.0248,  0.0044],
         ...,
         [-0.0029,  0.0076, -0.1719,  ...,  0.0215,  0.0110,  0.0073],
         [ 0.0